<a href="https://colab.research.google.com/github/Daku-on/signate_rag_2024/blob/main/src/01_glucose_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 23.9 MB/s eta 0:00:00


In [2]:
import datetime
import os
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np

In [3]:
NOW = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [4]:
# Hugging Faceの埋め込みモデルをロード
model_name = "pkshatech/GLuCoSE-base-ja"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/842k [00:00<?, ?B/s]

entity_vocab.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/867 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [5]:
# 1チャンクあたりのトークン数
MAX_TOKENS = 128

In [6]:
# 埋め込みベクトルを取得する関数
def get_embedding(text: str) -> np.ndarray:
    inputs = tokenizer(text,
                       return_tensors="pt",
                       truncation=True,
                       padding=True,
                       max_length=MAX_TOKENS)

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

    return embeddings

# テキストをチャンキングする関数
def chunk_text(text: str, max_tokens: int) -> list:
    tokens = tokenizer.tokenize(text)
    chunks = [' '.join(tokens[i:i+max_tokens]) for i in range(0, len(tokens), max_tokens)]
    return chunks

# フォルダ内の.txtファイルを読み込んでチャンキングし、ベクトルを取得する関数
def process_files_in_folder(folder_path: str) -> tuple:
    embeddings = []
    text_chunks = []  # 各チャンクされたテキストを保存

    # フォルダ内の全ファイルを取得
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()

                # テキストをチャンキング
                chunks = chunk_text(text, MAX_TOKENS)

                # 各チャンクに対して埋め込みベクトルを取得
                for chunk in chunks:
                    embedding = get_embedding(chunk)
                    embeddings.append(embedding)
                    text_chunks.append(chunk)  # チャンクされたテキストを保持

    return np.array(embeddings), text_chunks

# FAISSインデックスを保存する関数
def save_faiss_index_with_texts(index: faiss.Index, texts: list, index_file: str, text_file: str):
    # FAISSインデックスを保存
    faiss.write_index(index, index_file)

    # テキストチャンクをファイルに保存
    with open(text_file, 'w', encoding='utf-8') as f:
        for text in texts:
            f.write(text + "\n")

# FAISSインデックスを読み込む関数
def load_faiss_index_with_texts(index_file: str, text_file: str) -> tuple:
    # FAISSインデックスを読み込む
    index = faiss.read_index(index_file)

    # テキストチャンクを読み込む
    with open(text_file, 'r', encoding='utf-8') as f:
        texts = f.read().splitlines()

    return index, texts

# FAISSに追加して検索する関数
def search_faiss(query: str, index: faiss.Index, texts: list, top_k: int = 5):
    # クエリテキストの埋め込みを取得
    query_embedding = get_embedding(query).reshape(1, -1)

    # 類似検索
    D, I = index.search(query_embedding, top_k)

    # 検索結果のテキストと距離を返す
    return [(texts[idx], D[0][i]) for i, idx in enumerate(I[0])]



In [7]:
# 実行部分

# 1. ドキュメントを読み込み、FAISSインデックスを作成して保存
folder_path = "/content/drive/MyDrive/signate_rag_2024/novels"  # テキストファイルがあるフォルダ
index_file = f"/content/drive/MyDrive/signate_rag_2024/faiss_index/faiss_index_20240913-133606.index"  # FAISSインデックスの保存先
text_file = f"/content/drive/MyDrive/signate_rag_2024/faiss_index/texts_20240913-133606.txt"  # テキストチャンクの保存先



In [8]:
# embeddings, text_chunks = process_files_in_folder(folder_path)
# # FAISSインデックスの作成
# index = faiss.IndexFlatL2(embeddings.shape[1])
# index.add(embeddings)

# # インデックスとテキストチャンクを保存
# save_faiss_index_with_texts(index, text_chunks, index_file, text_file)

In [9]:
# 2. 保存されたFAISSインデックスとテキストを読み込み、検索を実行
index, loaded_text_chunks = load_faiss_index_with_texts(index_file, text_file)

# クエリを検索
query_text = "「死生に関するいくつかの断想」に出てくる最初の日付を教えてください"
results = search_faiss(query_text, index, loaded_text_chunks)

# 結果を表示
print(f"Query: {query_text}")
for i, (text, distance) in enumerate(results):
    print(f"Result {i+1}: {text} (Distance: {distance})")

Query: 「死生に関するいくつかの断想」に出てくる最初の日付を教えてください
Result 1: ▁[ # ここから 1 段階 小さな 文字 ] ▁注 ▁(1) イル カの 頭 のような 格好 をした 木の 塊 で 、 中 が空 洞 になっている 。 仏教 の 読 経 に合わせて 敲 かれる 。 ▁[ # ここで 小さな 文字 終わり ] ▁[ # ここで 字 下げ 終わり ] ▁[ # 19 字 下げ ] 3 [ # 「3 」 は 中 見出し ] ▁八 月 二 九 日 。 ▁ある 仏教 宗 派の 葬儀 の 儀式 に従って 、 遺 体が 火 葬 され るとき 、 骨 の中から 、 ほと け さん 、 もしくは 「 仏 さま 」 と呼ばれる 小さな 骨が 探 される 。 これは 、 一般 には 喉 の 小さな 骨 である と考えられている 。 どの 骨が それ なのか 、 私は 分からない し 、 また 、 そのような 遺 骨 を調べ (Distance: 83.0018310546875)
Result 2: ▁ 死 生 に関する いくつかの 断 想 ▁B IT S ▁O F ▁LI FE ▁A N D ▁D E AT H ▁小 泉 八 雲 ▁L af c ad io ▁H ear n ▁ 林 田 清 明 訳 ▁- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- ▁【 テキスト 中に 現れる 記号 について 】 ▁《 》 : ル ビ ▁( 例 ) 御 幣 《 ご へ い 》 ▁| : ル ビ の 付く 文字列 の 始まり を特定 する 記号 ▁( 例 ) 杉 本 | 嘉 作 《 か さく 》 ▁[ # ] : 入力 者 注 ▁主に 外 字 の説明 や 、 傍 点 (Distance: 83.16188049316406)
Result 3: となる ものであった ろう (1) [ # 「 (1 )」 は行 右 小 書き ]。 ▁[ # ここから 3 字 下げ ] ▁[ # ここから 1 段階 小さな 文字 ] ▁注 ▁(1) 大阪の 天王 寺 という 大きな お寺 では 、 この 骨 は みんな 納 骨 所に 投げ 込まれる 。 骨が 「 落ち ると

In [10]:
!pip install openai langchain tiktoken langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.

In [11]:
import datetime
import os
import openai
import pandas as pd
import tiktoken
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_core.prompts.prompt import PromptTemplate

In [12]:
from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

In [13]:
# OpenAI LLMの設定
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=openai.api_key
)

<ipython-input-13-a1e9d32a5811>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


In [14]:
# promptのテンプレート
template = """
質問に対して、以下の情報をもとに回答してください。
また、わからない場合には「わかりません。」と回答してください。
\n
質問: {question}\n
情報: {context}
"""
prompt = PromptTemplate(
    template=template,
    input_variables=[
        "question",
        "context",
    ]
)

In [25]:
def faiss_retriever(
    query: str,
) -> str:
    results = search_faiss(query, index, loaded_text_chunks)
    all_contexts = "\n".join([text for text, _ in results])
    return all_contexts

In [26]:
# 検索と生成を統合したチェーンの作成
qa_chain = (
    {
        "context": faiss_retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

In [17]:
qa_chain.invoke("「死生に関するいくつかの断想」に出てくる最初の日付を教えてください").content

'「死生に関するいくつかの断想」に出てくる最初の日付は「八月二十九日」です。'

In [27]:
faiss_retriever("骸骨男の正体")

'その 顔 ! ▁やっぱり そうでした 。 骸 骨 です 。 骸 骨の 顔 です 。 ▁ 骸 骨 男は 、 じゅう たん の中に かく れて 、 しの び こ んだ のです 。 なん という 、 うまい か くれ 場所 でしょう 。 そ と からは 、 三 まい の大きな じゅう たん が 、 か たく 巻 いて ある ように見 えますが 、 中は 、 人間 ひとり 、 横 になれる ほどの 空 洞 になっていた のです 。 ▁ま っ 黒 な 骸 骨 男は 、 廊 下の 壁 をつ たって 、 奥 のほう へ しの び こ んでいきます 。 食堂 の前 を とお って 、 台 所 へ 。 しかし 、 食堂 にいた おお ぜい の人は だれ も 気が つき ません 。 ▁ああ 、 ぶ き みな 骸 骨 男は 、 いったい\nていました 。 骸 骨 男が 、 なぜ こんな 曲 芸 をや っているのか 、 その 気 もち が わかりません 。 ▁ ぐる ぐる まわ っている ゾ ウ が 、 楽 屋 口 の前 を とお りました 。 そのとき 、 頭 の上に 立っている 骸 骨 男の 顔に 、 お ど ろ きの 色が あら われました 。 骸 骨 男の 目と 小林 少年の 目と が 、 ぶ っつ かった のです 。 骸 骨 男は 、 そのときは じめて 、 テント の中に 小林 君の いる ことを し りました 。 ▁ 骸 骨 男の 笑 いが と まりました 。 そして 、 恐ろしい 声が ひび いて きました 。 ▁「 そこ にいる のは 、 明 智 の 弟 子の 小林 だな ッ 。」 ▁ 走 っていた ゾ ウ が立ち ど まりました 。 骸 骨 男が と\n骸 骨 男に 化け ていた なんて 、 だれ も 、 考え つかない ことでした 。 そこに 、 あなたの 恐ろしい 秘密 があった のです 。 ▁いつも 、 骸 骨 男が 消えた あとに 、 あなたが あら われています 。 しかし 、 だれ もう たが わなかった のです 。 あなたと 骸 骨 男 と同じ 人だ なんて 、 どうして 想像 できるでしょう 。 ▁いつか 、 大型 バス の中から 骸 骨 男が 消えた のも 、 バスの 床に 、 かく し 戸 がつ いていた というのは ごま かし で 、 じ つは 

In [28]:
# 提供されたCSVファイルを読み込み
query_df = pd.read_csv("/content/drive/MyDrive/signate_rag_2024/data/query.csv", encoding="utf-8")

In [29]:
# 各問題に対して回答と証拠を取得する関数
def get_answer_and_evidence(problem):
    result = qa_chain.invoke(problem)
    answer = result.content
    evidence = faiss_retriever(problem) # 証拠部分を抽出
    return answer, evidence

# 各行の問題に対して処理を実行し、回答と証拠を取得
answers = []
evidences = []

for _, row in query_df.iterrows():
    problem = row['problem']
    answer, evidence = get_answer_and_evidence(problem)
    answers.append(answer)
    evidences.append(evidence)

In [30]:
# DataFrameに回答と証拠を追加
query_df['answer'] = answers
query_df['full_evidence'] = evidences
query_df.head()

,index,problem,answer,full_evidence
0,1,競漕会の三日前のレースコースでの結果は、農科と文科でどれくらいの秒数差があったか？,競漕会の三日前のレースコースでの結果について、農科と文科の秒数差は明記されていません。しかし...,昨日 久 野 が 潜 んでいた あたりは 、 今日は 夕方 から 曇 ったので ただ 茫 《...
1,2,骸骨男の正体は誰ですか？作中で言及されている氏名で答えること。,わかりません。,聞くと 、 顔 見 あわせて だ まり こんで しまいました 。 ▁ 骸 骨 男は 人間 に...
2,3,骸骨男はバスの中に足跡を一切残さずにどうやって抜け出しましたか？,わかりません。,骸 骨 男に 化け ていた なんて 、 だれ も 、 考え つかない ことでした 。 そこに...
3,4,殺人罪で裁判にかけられた兄が登場しますが、その理由は何ですか？,兄が殺人罪で裁判にかけられた理由についての情報は提供されていないため、具体的な理由はわかりません。,夜 、 土 の中から 小 判 を 拾 い 上げて 、 家に 持ち 帰った 。 その後 、 近...
4,5,小説「芽生」で出てくる国名は何種類ですか？,わかりません。,▁ 芽 生 ▁島 崎 藤 村 ▁- -- -- -- -- -- -- -- -- -- -...


In [31]:
# LLMを使ってanswerに基づき、evidenceから200文字程度を抜き出す関数
def extract_relevant_evidence(answer, full_evidence):
    extract_prompt = PromptTemplate(
        input_variables=["answer", "full_evidence"],
        template=
            """
                以下は回答と関連する文章です。
                回答に必要な部分を200文字以内で抜き出してください。要約した文章のみ回答してください\n
                回答: {answer}\n\n
                文章: {full_evidence}\n
            """
    )
    chain = extract_prompt | llm

    response = chain.invoke(
        {"answer": answer, "full_evidence": full_evidence}
    )
    return response.content

In [32]:
# full_evidenceから関連する部分を抜き出す
query_df['evidence'] = query_df.apply(
    lambda row: extract_relevant_evidence(row['answer'], row['full_evidence']),
    axis=1
)

In [33]:
# evidenceがNULLの箇所については「わかりません」で埋める
query_df['evidence'].fillna("わかりません。")

0     競漕会の三日前、文科は五分十五秒、農科は五分二十秒以上かかった。文科の方が速かったため、秒数...
1                                               わかりません。
2     骸骨男に化けていたのはあなたで、いつも消えた後に現れていた。大型バスの中でのトリックや、吉十...
3     兄が殺人罪で裁判にかけられた理由は不明であるが、彼は二〇歳の時に恋愛関係にあった女性との結婚...
4                                               わかりません。
5     武男は浪子に呼びかけられ、思わず身を震わせる。浪子は武男を見つめ、赤面しながらも無言でいる。...
6     小説「芽生」では、主人公が東京から小諸に帰る時期から引越しが完了するまでの具体的な日数は明示...
7     文科のボート部メンバーが夕飯時に特別に許可された1回のシーンで、約2合の酒を皆で飲んだことが...
8           文中に登場するカタカナ表記された北海道の地名は以下の通りです。マッカリヌプリ、内浦湾。
9     彼は決して一杯の酒すらも飲まなかったし、独身のままであった。兄は二〇歳の頃、ある旅館の綺麗な...
10                                              わかりません。
11    水玉もようの道化師が空中サーカスの名人たちを呼び、肉じゅばんに金糸のぬいとりのあるサルマタを...
12    より健康だったのは歳下の兄弟で、彼は人力車夫として一家を支えていた。長男は三四歳で亡くなり、...
13    笠原正一君が見た恐ろしい夢は、薄暗い空から豆つぶのようなものが降り、それが次第に大きくなり、...
14    毛がわの腹には隠しボタンがついていて、人間が出入りできるようになっていた。二十面相はそのボタ...
15    柱と柱との間に、濃紺、紫、バラ色、薄青、パールグレイなど、さまざまな色の絹や綿の長い布を広げ...
16    主人公が「最後の晩餐」と思った理由についての具体的な情報は提供されておらず、詳細を読み取るこ...
17    中村警部、明智、小林、井上、笠原、野呂の名前が登場し、少年探偵団が怪事件に取り組む様

In [34]:
# answerとevidenceカラムにある改行コードを削除する
replace_dict = {
        "\n": "",
        "\r": "",
    }
query_df = query_df.replace(
        {"answer": replace_dict},
        regex=True
    )
query_df = query_df.replace(
        {"evidence": replace_dict},
        regex=True
    )

# 要約

In [35]:
# LLMを使って要約を行う関数
def summarize_answer(answer: str) -> str:
    # OpenAI API などの LLM を使用して要約を実行

    summarize_prompt = PromptTemplate(
        input_variables=["answer"],
        template=
            """
                以下の文章を50文字程度で要約してください。\n
                f"回答: {answer}"
            """
    )
    chain = summarize_prompt | llm

    response = chain.invoke(
        {"answer": answer}
    )
    return response.content

# tiktokenとgpt-4のトークナイザーを取得
enc = tiktoken.encoding_for_model("gpt-4-2024-08-06")

# query_df の "answer" 列のトークン数を計算し、50トークンを超える場合は要約を行う関数
def check_and_summarize_answers(query_df: pd.DataFrame) -> pd.DataFrame:
    def summarize_if_needed(answer: str) -> str:
        # トークン数を計算
        token_count = len(enc.encode(answer))

        # トークン数が50を超えた場合は要約する
        while token_count > 50:
            answer = summarize_answer(answer)
            token_count = len(enc.encode(answer))
        return answer

    # "answer" 列に対して処理を適用
    query_df["answer"] = query_df["answer"].apply(summarize_if_needed)
    return query_df

In [36]:
# check_and_summarize
query_df = check_and_summarize_answers(query_df)

In [37]:
# 必要な列（id, answer, evidence）をヘッダなしでCSVに書き出し
query_df[['index', 'answer', 'evidence']].to_csv(
    "/content/drive/MyDrive/signate_rag_2024/data/evaluation/submit/predictions.csv",
    index=False,
    header=False,
    encoding="utf-8-sig"
)

In [39]:
# バックアップ用に別のファイルにも保存
NOW = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

query_df[['index', 'answer', 'evidence']].to_csv(
    f"/content/drive/MyDrive/signate_rag_2024/data/evaluation/submit/predictions_{NOW}.csv",
    index=False,
    header=False,
    encoding="utf-8-sig"
)